In [1]:
import sys
sys.path.insert(0, '../../')

import os
import numpy as np
from tqdm import tqdm


import torch
from torch.utils import data
import torchvision.transforms as transform
from torch.nn.parallel.scatter_gather import gather

import encoding.utils as utils
from encoding.nn import SegmentationLosses, SyncBatchNorm
from encoding.parallel import DataParallelModel, DataParallelCriterion
from encoding_custom.datasets import get_segmentation_dataset, test_batchify_fn
from encoding.models import get_model, MultiEvalModule
from encoding_custom.models import get_segmentation_model\

import json
from experiments.segmentation.option import Options
import matplotlib.pyplot as plt
import glob
from PIL import Image

%matplotlib inline

In [5]:
test_folder = '/volumes2/cpp/predicts_1024/'
tet_folder_gt = '/data/input/datasets/Mapillary_v1.1/validation/labels/'
test_labels = glob.glob(os.path.join(tet_folder_gt, '*.png'))
num_class = 65
print(len(test_labels))

2000


In [6]:
def crop(img, factor):
    height_crop = (img.shape[0] // factor) * factor
    width_crop = (img.shape[1] // factor) * factor

    diff_hight = img.shape[0] - height_crop
    diff_width = img.shape[1] - width_crop

    offset_height = diff_hight // 2
    offset_width = diff_width // 2

    return img[
          offset_height:height_crop + offset_height,
          offset_width:width_crop + offset_width]



In [7]:
total_inter, total_union, total_correct, total_label, total_loss = 0, 0, 0, 0, 0
tbar = tqdm(test_labels)

for label_image in tbar:
    im_label = Image.open(label_image)
    file_path = os.path.join(test_folder,label_image.split('/')[-1][:-3]+'dat')
    assert(os.path.exists(file_path))

    predicts_cpp = np.fromfile(file_path, dtype=np.byte, count=im_label.size[0]*im_label.size[1])
    predicts_cpp = np.reshape(predicts_cpp, im_label.size[::-1])
    
    short_size = 1024
    w, h = im_label.size
    if w > h:
        oh = short_size
        ow = int(1.0 * w * oh / h)
    else:
        ow = short_size
        oh = int(1.0 * h * ow / w)
    
    im_label = im_label.resize((ow, oh), Image.NEAREST)

    predicts_cpp = Image.fromarray(predicts_cpp)
    predicts_cpp = predicts_cpp.resize((ow, oh), Image.NEAREST)

    im_label = np.asarray(im_label)
    predicts_cpp = np.asarray(predicts_cpp)
    
    im_label = crop(im_label, 32)
    predicts_cpp = crop(predicts_cpp, 32)
    
    correct, labeled  = utils.metrics.pixel_accuracy(predicts_cpp +1 , im_label + 1)
    inter, union = utils.metrics.intersection_and_union(predicts_cpp + 1, im_label + 1, num_class)
    
    total_inter += inter
    total_union += union
    total_correct += correct
    total_label += labeled
    
    pixAcc = 1.0 * total_correct / (np.spacing(1) + total_label)
    IoU = 1.0 * total_inter / (np.spacing(1) + total_union)
    mIoU = IoU.mean()
    tbar.set_description( 'pixAcc: %.4f, mIoU: %.4f' % (pixAcc, mIoU))


pixAcc: 0.8927, mIoU: 0.3915: 100%|██████████| 2000/2000 [05:18<00:00,  6.11it/s]
